In [2]:
using DataFrames
using GLM

In [3]:
train=readtable("./train.csv")
store=readtable("./store.csv")
test=readtable("./test.csv")

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
1,1,1,4,2015-09-17,1,1,0,0
2,2,3,4,2015-09-17,1,1,0,0
3,3,7,4,2015-09-17,1,1,0,0
4,4,8,4,2015-09-17,1,1,0,0
5,5,9,4,2015-09-17,1,1,0,0
6,6,10,4,2015-09-17,1,1,0,0
7,7,11,4,2015-09-17,1,1,0,0
8,8,12,4,2015-09-17,1,1,0,0
9,9,13,4,2015-09-17,1,1,0,0
10,10,14,4,2015-09-17,1,1,0,0


In [ ]:
function center(df, colName)
    return df[colName] .- mean(train[colName])
end
# train[:Customers] = center(train, :Customers)
# test[:Customers] = center(test, :Customers)

In [4]:
function stringToBool(holidayCol)
    data = DataArray(Bool, length(holidayCol))
    for i=1:size(holidayCol,1)
        data[i] = ("0" != holidayCol[i])
    end
    return data
end
train[:StateHoliday] = stringToBool(train[:StateHoliday])
test[:StateHoliday] = stringToBool(test[:StateHoliday])
train[:SchoolHoliday] = convert(DataArray{Bool,1}, train[:SchoolHoliday])
test[:SchoolHoliday] = convert(DataArray{Bool,1}, test[:SchoolHoliday])
train[:Open] = convert(DataArray{Bool,1}, train[:Open])
test[:Open] = convert(DataArray{Bool,1}, test[:Open])

41088-element DataArrays.DataArray{Bool,1}:
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
    ⋮
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true

# Poisson Regression

In [ ]:
fitByStoreOpen = by(
    train[train[:Open] .== true, :],
    :Store,
    df -> glm(Sales~DayOfWeek+Promo+SchoolHoliday, df, Poisson())
)
fitByStore = by(
    train,
    :Store,
    df -> glm(Sales~DayOfWeek+Promo+SchoolHoliday, df, Poisson())
)

In [ ]:
function makePreds(df)
    preds = DataArray(Float64, size(df,1))
    preds[!isna(df[:Open]) & (df[:Open] .== false)] = 0
    for i=unique(df[:Store])
        modelKnownOpen = fitByStoreOpen[fitByStoreOpen[:Store] .== i, :x1][1]
        modelUnknownOpen = fitByStore[fitByStore[:Store] .== i, :x1][1]
        mask = !isna(df[:Open]) & (df[:Open] .== true) & (df[:Store] .== i)
        preds[mask, :] = predict(modelKnownOpen, df[mask, :])
        mask = isna(df[:Open]) & (df[:Store] .== i)
        preds[mask, :] = predict(modelUnknownOpen, df[mask, :])
    end
    return preds
end

In [ ]:
trainPreds = makePreds(train)
mean(abs(makePreds(train) - preds))

In [ ]:
testPreds = makePreds(test)

In [ ]:
writetable("output.csv", DataFrame(Id=1:length(testPreds), Sales=testPreds))

# Random Forest Regression

In [5]:
using DecisionTree

In [7]:
labels=convert(Array{Float64}, train[:Sales])
features=convert(Array{Float64}, train[[:Store, :DayOfWeek, :Promo, :SchoolHoliday, :StateHoliday]])

1017209x5 Array{Float64,2}:
    1.0  5.0  1.0  1.0  0.0
    2.0  5.0  1.0  1.0  0.0
    3.0  5.0  1.0  1.0  0.0
    4.0  5.0  1.0  1.0  0.0
    5.0  5.0  1.0  1.0  0.0
    6.0  5.0  1.0  1.0  0.0
    7.0  5.0  1.0  1.0  0.0
    8.0  5.0  1.0  1.0  0.0
    9.0  5.0  1.0  1.0  0.0
   10.0  5.0  1.0  1.0  0.0
   11.0  5.0  1.0  1.0  0.0
   12.0  5.0  1.0  1.0  0.0
   13.0  5.0  1.0  0.0  0.0
    ⋮                      
 1104.0  2.0  0.0  1.0  1.0
 1105.0  2.0  0.0  1.0  1.0
 1106.0  2.0  0.0  1.0  1.0
 1107.0  2.0  0.0  1.0  1.0
 1108.0  2.0  0.0  1.0  1.0
 1109.0  2.0  0.0  1.0  1.0
 1110.0  2.0  0.0  1.0  1.0
 1111.0  2.0  0.0  1.0  1.0
 1112.0  2.0  0.0  1.0  1.0
 1113.0  2.0  0.0  1.0  1.0
 1114.0  2.0  0.0  1.0  1.0
 1115.0  2.0  0.0  1.0  1.0

In [37]:
model = build_forest(labels, features, 2, 50, 0.5)

Ensemble of Decision Trees
Trees:      50
Avg Leaves: 185.42
Avg Depth:  16.48

In [41]:
function makePredsRf(df, model)
    preds = DataArray(Float64, size(df,1))
    closedAndNotNA = !isna(df[:Open]) & (df[:Open] .== false)
    preds[closedAndNotNA] = 0
    preds[!closedAndNotNA, :] = apply_forest(
        model,
        convert(
            Array{Float64},
            df[!closedAndNotNA, [:Store, :DayOfWeek, :Promo, :SchoolHoliday, :StateHoliday]]))
    return preds
end

makePredsRf (generic function with 1 method)

In [42]:
trainPreds = makePredsRf(train, model)
mean(abs(train[:Sales] - trainPreds))

1666.9589104069817

In [43]:
testPreds = makePredsRf(test, model)
writetable("output_rf.csv", DataFrame(Id=test[:Id], Sales=testPreds))